In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('Elephas_App').setMaster('local[8]')
try:
    sc.stop()
except:
    pass
sc = SparkContext(conf=conf)

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Flatten, Dropout
from keras.layers.merge import Dot, multiply, concatenate
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import skipgrams
from keras import losses
from keras import metrics

Using TensorFlow backend.


In [34]:
def get_model(max_work, max_user):
    dim_embedddings = 30
    bias = 1
    # inputs
    w_inputs = Input(shape=(1,), dtype='int32')
    w = Embedding(max_work+1, dim_embedddings, name="work")(w_inputs)
    w_bis = Embedding(max_work + 1, bias, name="workbias")(w_inputs)

    # context
    u_inputs = Input(shape=(1,), dtype='int32')
    u = Embedding(max_user+1, dim_embedddings, name="user")(u_inputs)
    u_bis = Embedding(max_user + 1, bias, name="userbias")(u_inputs)
    
    o = multiply([w, u])
    o = Dropout(0.5)(o)
    o = concatenate([o, u_bis, w_bis])
    o = Flatten()(o)
    o = Dense(10, activation="relu")(o)
    o = Dense(1)(o)

    rec_model = Model(inputs=[w_inputs, u_inputs], outputs=o)
    #rec_model.summary()
    rec_model.compile(loss = losses.mean_squared_error, optimizer='adam', metrics=[metrics.mae])

    return rec_model

In [5]:
def get_data(path):
    data = pd.read_csv(path)

    mapping_work = get_mapping(data["movieId"])

    data["movieId"] = data["movieId"].map(mapping_work)

    mapping_users = get_mapping(data["movieId"])

    data["movieId"] = data["movieId"].map(mapping_users)

    percentil_80 = np.percentile(data["timestamp"], 80)

    print(percentil_80)

    print(np.mean(data["timestamp"]<percentil_80))

    print(np.mean(data["timestamp"]>percentil_80))

    cols = ["userId", "movieId", "rating"]

    train = data[data.timestamp<percentil_80][cols]

    print(train.shape)

    test = data[data.timestamp>=percentil_80][cols]

    print(test.shape)

    max_user = max(data["userId"].tolist() )
    max_work = max(data["movieId"].tolist() )


    return train, test, max_user, max_work, mapping_work

In [6]:
from collections import defaultdict
def get_mapping(series):
    occurances = defaultdict(int)
    for element in series:
        occurances[element] += 1
    mapping = {}
    i = 0
    for element in occurances:
        i += 1
        mapping[element] = i

    return mapping

In [7]:
def get_array(series):
    return np.array([[element] for element in series])

In [9]:
train, test, max_user, max_work, mapping_work = get_data("../data/ml-latest-1m/ratings.csv")

975768738.0
0.7999968006686603
0.19999920016716508
(800164, 3)
(200045, 3)


In [35]:
model = get_model(max_work, max_user)

In [36]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_25 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
work (Embedding)                (None, 1, 30)        111210      input_24[0][0]                   
__________________________________________________________________________________________________
user (Embedding)                (None, 1, 30)        181230      input_25[0][0]                   
__________________________________________________________________________________________________
multiply_8

In [4]:
sql_password="Myiphone5s"
df=sqlContext.read.format("jdbc").options(url="jdbc:mysql://localhost:3306/MovieDB",
                                       driver="com.mysql.jdbc.Driver",
                                       dbtable="movies",user="root",password=sql_password).load()

In [5]:
prop = {"driver":"com.mysql.jdbc.Driver"}

rating_df = sqlContext.read.format("jdbc").options(
    url = "jdbc:mysql://localhost:3306/MovieDB?user=root&password="+sql_password,
    dbtable = "ratings", properties = prop).load()

ratings = rating_df.select('userId','movieId','rating').rdd

splits = ratings.randomSplit([0.8,0.2],10)
train_data = splits[0].repartition(3) #分区
val_data = splits[1].repartition(3)

In [12]:
x_train = [get_array(train["movieId"]), get_array(train["userId"])]
y_train = get_array(train["rating"])

In [37]:
from elephas.utils.rdd_utils import to_simple_rdd
rdd = to_simple_rdd(sc, x_train, y_train)

In [39]:
from elephas.spark_model import SparkModel

spark_model = SparkModel(model, frequency='epoch', mode='asynchronous')
spark_model.fit(rdd, epochs=20, batch_size=32, verbose=0, validation_split=0.1)

>>> Fit model
 * Serving Flask app "elephas.parameter.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 1.0 failed 1 times, most recent failure: Lost task 6.0 in stage 1.0 (TID 14, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.4.0/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 359, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/usr/local/Cellar/apache-spark/2.4.0/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 64, in read_command
    command = serializer._read_with_length(file)
  File "/usr/local/Cellar/apache-spark/2.4.0/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 172, in _read_with_length
    return self.loads(obj)
  File "/usr/local/Cellar/apache-spark/2.4.0/libexec/python/lib/pyspark.zip/pyspark/serializers.py", li

In [13]:
history = model.fit(x_train, y_train, nb_epoch=10,
                    validation_split=0.2, verbose=1)

Instructions for updating:
Use tf.cast instead.


/Users/ChongKaHung/anaconda3/envs/spark/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 640131 samples, validate on 160033 samples
Epoch 1/10
640131/640131 [==============================] - 125s 195us/step - loss: 1.0112 - mean_absolute_error: 0.7744 - val_loss: 1.0126 - val_mean_absolute_error: 0.8122
Epoch 2/10
640131/640131 [==============================] - 116s 182us/step - loss: 0.7943 - mean_absolute_error: 0.7021 - val_loss: 1.0245 - val_mean_absolute_error: 0.8235
Epoch 3/10
640131/640131 [==============================] - 120s 187us/step - loss: 0.7518 - mean_absolute_error: 0.6805 - val_loss: 1.0220 - val_mean_absolute_error: 0.8233
Epoch 4/10
640131/640131 [==============================] - 136s 213us/step - loss: 0.7220 - mean_absolute_error: 0.6650 - val_loss: 1.0350 - val_mean_absolute_error: 0.8311
Epoch 5/10
640131/640131 [==============================] - 122s 190us/step - loss: 0.7008 - mean_absolute_error: 0.6544 - val_loss: 1.0100 - val_mean_absolute_error: 0.8162
Epoch 6/10
640131/640131 [==============================] - 124s 194us/step - 

In [14]:
from sklearn.metrics import mean_absolute_error

In [15]:
predictions = model.predict([get_array(test["movieId"]), get_array(test["userId"])])

In [16]:
test_performance = mean_absolute_error(test["rating"], predictions)

In [17]:
test_performance

0.7667506844570315

In [ ]:
sc.stop()